# 01 - The Ariadne Advantage: Intelligent Routing Demonstration

This notebook showcases Ariadne's prioritized filter chain in action. We walk through two representative circuits:

1. A **low-entanglement, non-Clifford** chain that routes to the Matrix Product State (MPS) backend.
2. A **high-entanglement** circuit that is rejected by the specialized filters and falls back to the dense simulator.

Run each cell sequentially to reproduce the routing decisions and confidence scores.

> **Colab tip:** Before running, choose Runtime → Change runtime type → Python 3.11.


In [ ]:
# Install Ariadne (prefer release builds, fallback to latest main)
!pip install -q --upgrade "ariadne-router>=0.4.1,<0.5.0" \
        || pip install -q git+https://github.com/Hmbown/ariadne.git

In [ ]:
import numpy as np
from packaging.version import Version
from qiskit import QuantumCircuit

from ariadne import __version__, explain_routing, simulate

print(f"Ariadne version: {__version__}")
if Version(__version__) < Version("0.4.1"):
    raise SystemExit("Please upgrade ariadne-router to 0.4.1 or newer and rerun.")

## Case 1: Low Entanglement Circuit (MPS Candidate)

We build a 25-qubit nearest-neighbor chain with shallow depth. Lightweight `ry` rotations on alternating qubits break the Clifford structure while keeping entanglement growth minimal. The router should select the MPS backend with high confidence.


In [ ]:
# Low entanglement circuit (25 qubits, shallow depth, local interactions)
N_LOW = 25
qc_low = QuantumCircuit(N_LOW)
qc_low.h(range(N_LOW))
for i in range(0, N_LOW, 2):
    qc_low.ry(np.pi / 8, i)
for i in range(0, N_LOW - 1, 2):
    qc_low.cx(i, i + 1)
qc_low.measure_all()

print(f"Low entanglement circuit: {N_LOW} qubits, depth {qc_low.depth()}")
print("Routing explanation:")
print(explain_routing(qc_low))

In [ ]:
# Simulate the circuit
result_low = simulate(qc_low, shots=200)

print("\n--- Low Entanglement Circuit Routing ---")
print(f"Backend Selected: {result_low.backend_used.value}")
print(f"Execution Time: {result_low.execution_time:.4f}s")
print(f"Confidence Score: {result_low.routing_decision.confidence_score:.3f}")
print(f"Throughput: {200 / max(result_low.execution_time, 1e-6):.0f} shots/sec")
print(f"Sample results (first 5 states): {dict(list(result_low.counts.items())[:5])}")

## Case 2: High Entanglement Circuit (Dense Backend Candidate)

We now create a 10-qubit circuit with multiple non-local entangling layers. The entanglement growth invalidates the MPS heuristic, so the router falls back to the fastest dense backend available (for example, Qiskit Aer).


In [ ]:
# High entanglement circuit (10 qubits, deep, highly non-local)
N_HIGH = 10
qc_high = QuantumCircuit(N_HIGH)
for _ in range(5):  # 5 layers of entanglement
    for i in range(N_HIGH):
        qc_high.rx(np.pi / 4, i)
    for i in range(N_HIGH - 1):
        qc_high.cz(i, i + 1)
    qc_high.cz(0, N_HIGH - 1)  # Add a non-local edge to raise entanglement
    qc_high.barrier()
qc_high.measure_all()

print(f"High entanglement circuit: {N_HIGH} qubits, depth {qc_high.depth()}")
print("Routing explanation:")
print(explain_routing(qc_high))

In [ ]:
# Simulate the high entanglement circuit
result_high = simulate(qc_high, shots=200)

print("\n--- High Entanglement Circuit Routing ---")
print(f"Backend Selected: {result_high.backend_used.value}")
print(f"Execution Time: {result_high.execution_time:.4f}s")
print(f"Confidence Score: {result_high.routing_decision.confidence_score:.3f}")
print(f"Throughput: {200 / max(result_high.execution_time, 1e-6):.0f} shots/sec")
print(f"Sample results (first 5 states): {dict(list(result_high.counts.items())[:5])}")

## Summary

Ariadne's router correctly:

1. Analyzes circuit structure to estimate entanglement and sparsity.
2. Routes low-entanglement, non-Clifford circuits to the MPS backend.
3. Escalates highly entangled circuits to dense simulators automatically.
4. Provides transparent explanations and confidence scores for every decision.

Just write your quantum circuit — Ariadne handles backend selection, performance tuning, and routing transparency for you.
